# Housing market analysis

data
1. [Housing prices by quartiles](https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_destaques&DESTAQUESdest_boui=645849423&DESTAQUESmodo=2)
2. [CPI](https://www.ine.pt/xportal/xmain?xpid=INE&xpgid=ine_indicadores&contecto=pi&indOcorrCod=0002384&selTab=tab0)


reading
* https://en.wikipedia.org/wiki/2010%E2%80%932014_Portuguese_financial_crisis
* https://ahouseinlisbon.com/en/housing-prices-in-portugal-over-the-last-6-years

In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from PyPDF2 import PdfMerger

%matplotlib inline

In [2]:
fig_dir = 'figures'

In [3]:
# plt.style.use('seaborn-v0_8-dark')
# plt.rcParams["font.family"] = 'times new roman'

# plot formatting
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# grid lines
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.axis'] = 'both'
plt.rcParams['grid.alpha'] = 0.25
plt.rcParams['grid.linewidth'] = 2

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [4]:
# combine files
first = True
for n in range(1, 8):

    src = f'VABH1024_en/Table {n}.csv'

    tmp = pd.read_csv(src, header=1, index_col=0, encoding='latin-1', delimiter=';')
    tmp = tmp.iloc[1:]
    tmp = tmp[~tmp.index.isna()]
    tmp = tmp.dropna(axis=0, how='all')
    
    if first:
        df = tmp.copy()
        first = False
    else:
        df = pd.merge(df, tmp, how='outer', suffixes=('', '_duplicate'), left_index=True, right_index=True)
        
cols = [c for c in df if 'duplicate' not in c]
df = df[cols]

# clean columns
def clean_cols(c):
    
    c = c.lower()

    replace = {
        ' ' : '',
        '-a-' : 'a', 
        '-' : '_',
        'quartile' : '',
        '1st' : 'q1',
        '3rd' : 'q3',
        '¢' : 'o',
        'æ' : 'a',
        '£' : 'u',
        '\x87' : 'c',
        '\xa0' : 'a',
        '¡' : 'i',
    }

    for k, v in replace.items():
        c = c.replace(k, v)
    
    return c

df.columns = [clean_cols(c) for c in df]

# multi-index column
cols = df.columns.str.split('_')
cols = [c+['all'] if len(c) < 3 else c for c in cols]
cols = [[c[0], c[1], c[-1]] for c in cols]
cols = [c[::-1] for c in cols]
df.columns = pd.MultiIndex.from_tuples(cols)

# clean values
df = df.astype(str)
df = df.map(lambda x : x if x[0].isnumeric() else None)
df = df.dropna(axis=0, how='all')
df = df.astype(float)

# clean index
replace = {
    'jan' : '1',
    'fev' : '2',
    'mar' : '3',
    'abr' : '4',
    'mai': '5',
    'jun' : '6',
    'jul' : '7',
    'ago' : '8',
    'set' : '9',
    'out' : '10',
    'nov' : '11',
    'dez' : '12',
}
for k, v in replace.items():
    df.index = df.index.str.replace(k, v)


df.index = pd.DatetimeIndex(df.index)
df = df.sort_index()

In [5]:
def plotq(
        ax, sdf,
        format=dict(marker='o', fillstyle='none')
        ):
    
    '''Plot quartile data'''

    x, y = sdf.index, sdf['median']
    ax.plot(x, y, label='median', **format)

    x, y = sdf.index, sdf['q1']
    ax.plot(x, y, label='1st quartile', **format)

    x, y = sdf.index, sdf['q3']
    ax.plot(x, y, label='3rd quartile', **format)

    return ax

In [6]:
def annualize(ds, months=12):

    ds = ds.pct_change(months)
    
    return ((1+ds)**(12 / months))-1

In [ ]:
place = 'porto'
build = 'apartments'

sdf = df[build][place]
cond = sdf.index >= '2020-12'

series = {
    f'{place.title()} {build}: Euro per square meter' : sdf[cond],
    f'{place.title()} {build}: 12-month returns' : sdf.pct_change(12)[cond],
    f'{place.title()} {build}: annualized 2-year returns' : sdf.apply(annualize, args=(2*12,))[cond],
    f'{place.title()} {build}: annualized 5-year returns' : sdf.apply(annualize, args=(5*12,))[cond],
    f'{place.title()} {build}: annualized 10-year returns' : sdf.apply(annualize, args=(10*12,))[cond],
}

for i, (title, sdf) in enumerate(series.items()):

    fig, ax = plt.subplots(figsize=(10, 5))
    
    ax.set_title(title)
    ax = plotq(ax, sdf)

    ax.yaxis.minorticks_on()
    ax.set_xticks(sdf.index[::6], sdf.index.strftime('%Y-%b')[::6])

    ax.axvline(pd.to_datetime('2023'), c='k', ls=':')
    if ax.get_ylim()[0] < 0:
        ax.axhline(0, c='r', ls='--')
    

    fig.legend()
    
    file = title.lower()
    for c in '-:':
        file = file.replace(c, '')
    file = file.replace(' ', '_')

    dst = os.path.join(fig_dir, f'{i}-{file}.pdf')
    fig.savefig(dst, bbox_inches=False)
    print('Saved:', dst)

    plt.show()

In [8]:
merger = PdfMerger()

files = os.listdir(fig_dir)
files = sorted(a for a in files if a.endswith(".pdf"))

for file in files:
    src = os.path.join(fig_dir, file)
    merger.append(open(src, 'rb'))

with open("figures.pdf", "wb") as f:
    merger.write(f)
